# Experiment using model 1
Description

In [45]:
import numpy as np
import pandas as pd
#import miguellib
#from miguellib.datasets.dummy import dummy_dataset

from pathlib import Path

print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
#print(f"Miguellib version: {miguellib.__version__}")
      

Numpy version: 2.1.3
Pandas version: 2.2.3


## Model theoretical explanation

This model is based in ...

In [46]:
# Parameters
batch_size = 32
n_layers = 5
#epochs = 10
EPOCHS = 10

## Data prep


Dataset Name: Comparing Cosmetics By Ingredients

Kaggle Link: https://www.kaggle.com/datasets/venessagreen/comparing-cosmetics-by-ingredients?resource=download

In [47]:
BASE_DIR = Path().resolve() 
DATASET_PATH = BASE_DIR / ".." / ".." / "MiguelLib" / "datasets" / "cosmetics.csv"
DATASET_PATH = DATASET_PATH.resolve()


df = pd.read_csv(DATASET_PATH)
df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [48]:
print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nData types:")
print(df.dtypes)
print("\nMissing values per column:")
print(df.isnull().sum())

Dataset shape: (1472, 11)

Columns: ['Label', 'Brand', 'Name', 'Price', 'Rank', 'Ingredients', 'Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']

Data types:
Label           object
Brand           object
Name            object
Price            int64
Rank           float64
Ingredients     object
Combination      int64
Dry              int64
Normal           int64
Oily             int64
Sensitive        int64
dtype: object

Missing values per column:
Label          0
Brand          0
Name           0
Price          0
Rank           0
Ingredients    0
Combination    0
Dry            0
Normal         0
Oily           0
Sensitive      0
dtype: int64


### Columns Overview

In [49]:
print("\n--- Product Categories (Label) ---")
category_counts = df["Label"].value_counts()
print(category_counts)
print("\nNumber of unique categories:", df["Label"].nunique())


--- Product Categories (Label) ---
Label
Moisturizer    298
Cleanser       281
Face Mask      266
Treatment      248
Eye cream      209
Sun protect    170
Name: count, dtype: int64

Number of unique categories: 6


In [54]:
print("\n--- Product Brands ---")
category_counts = df["Brand"].value_counts()
print(category_counts)
print("\nNumber of unique brands:", df["Brand"].nunique())


--- Product Brands ---
Brand
CLINIQUE              79
SEPHORA COLLECTION    66
SHISEIDO              63
ORIGINS               54
MURAD                 47
                      ..
SON & PARK             1
MAKEUP ERASER          1
KAT VON D              1
NURSE JAMIE            1
DERMAFLASH             1
Name: count, Length: 116, dtype: int64

Number of unique brands: 116


In [51]:
print("\n--- Price Overview ---")
print(df["Price"].describe())


--- Price Overview ---
count    1472.000000
mean       55.584239
std        45.014429
min         3.000000
25%        30.000000
50%        42.500000
75%        68.000000
max       370.000000
Name: Price, dtype: float64


In [55]:
print("\n--- Rank Overview ---")
print(df["Rank"].describe())


--- Rank Overview ---
count    1472.000000
mean        4.153261
std         0.633918
min         0.000000
25%         4.000000
50%         4.300000
75%         4.500000
max         5.000000
Name: Rank, dtype: float64


In [56]:
print("\n--- Ingredients Overview ---")
print(df["Ingredients"].head(5))
ingredient_lengths = df["Ingredients"].str.len()
print("\nIngredient text lengths stats:")
print(ingredient_lengths.describe())
print("\nRandom sample of ingredients:")
print(df["Ingredients"].sample(10, random_state=42))


--- Ingredients Overview ---
0    Algae (Seaweed) Extract, Mineral Oil, Petrolat...
1    Galactomyces Ferment Filtrate (Pitera), Butyle...
2    Water, Dicaprylyl Carbonate, Glycerin, Ceteary...
3    Algae (Seaweed) Extract, Cyclopentasiloxane, P...
4    Water, Snail Secretion Filtrate, Phenyl Trimet...
Name: Ingredients, dtype: object

Ingredient text lengths stats:
count    1472.000000
mean      700.720788
std       531.932518
min         6.000000
25%       374.500000
50%       648.000000
75%       933.250000
max      5491.000000
Name: Ingredients, dtype: float64

Random sample of ingredients:
852     Galactomyces Ferment Filtrate*, Water, Glyceri...
184     Adv Timezone Night Age Rvr Lw Crm Division: El...
1261    -Nutrex EGF -Epidermosil™ -Eye Regener™ -Diamo...
67      Water, Dimethicone, Butylene Glycol, Phenyl Tr...
220     Water , Glycerin , Cetyl Alcohol , Dimethicone...
494     Water, Hamamelis Virginiana (Witch Hazel) Wate...
430     Water , Isopropyl Palmitate , Pentylene G

In [52]:
skin_columns = ["Combination", "Dry", "Normal", "Oily", "Sensitive"]
print("\n--- Skin Type Flags Summary ---")
print(df[skin_columns].sum())
print("\nDistribution (percent) of skin flags:")
print((df[skin_columns].sum() / df.shape[0] * 100).round(2))


--- Skin Type Flags Summary ---
Combination    966
Dry            904
Normal         960
Oily           894
Sensitive      756
dtype: int64

Distribution (percent) of skin flags:
Combination    65.62
Dry            61.41
Normal         65.22
Oily           60.73
Sensitive      51.36
dtype: float64


In [44]:
zero_ingredients = (df["Ingredients"].str.strip() == "").sum()
print(f"\nNumber of products with empty ingredient list: {zero_ingredients}")


Number of products with empty ingredient list: 0


In [53]:
duplicates = df.duplicated(subset=["Brand", "Name"]).sum()
print(f"\nNumber of duplicate products by Brand+Name: {duplicates}")


Number of duplicate products by Brand+Name: 0


### Quick Dataset Insights

Dataset Overview:

- Rows × Columns: 1472 × 11

- Columns: Label, Brand, Name, Price, Rank, Ingredients, Combination, Dry, Normal, Oily, Sensitive

- Data types: Mixed; numerical (Price, Rank, skin-type flags) and categorical/text (Label, Brand, Name, Ingredients)

- Missing values: None

Product Categories (Label):

- 6 unique categories: Moisturizer (298), Cleanser (281), Face Mask (266), Treatment (248), Eye cream (209), Sun protect (170)

- Categories are fairly balanced, with Moisturizers being the most frequent

Brands:

- 116 unique brands, with top brands: CLINIQUE (79), SEPHORA COLLECTION (66), SHISEIDO (63), ORIGINS (54), MURAD (47)

- Long tail distribution: many brands with only 1–2 products

Price Overview:

- Prices are integers (currency not specified)

- Distribution is not detailed here but can be further explored

Rank Overview:

- Mean rank: 4.15 ± 0.63

- Range: 0 to 5 (some products have rank 0, may require inspection)

Ingredients:

- Text field with varied lengths: 6–5491 characters, mean ~701 characters

- No products have empty ingredient lists

- Some entries contain detailed ingredient lists, others include additional notes or boutique info – may need cleaning

Skin Type Flags:

- Multiple flags per product are possible

- Most common flags: Combination (65.6%), Normal (65.2%), Dry (61.4%), Oily (60.7%), Sensitive (51.4%)

Duplicates & Integrity:

- No duplicate products by Brand+Name

- All key fields populated

## Model training

In [ ]:
# model.fit

In [ ]:
# preds = model.test

## Evaluation

In [ ]:
# model_auc = auc(true, preds)

## Summary